In [1]:
# !pip install pyyaml==5.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
!python3 -m pip install ../input/nflimpactdetectron/detectron2/detectron2 --no-index --find-links ../input/nflimpactdetectron/wheelz/wheelz

Looking in links: ../input/nflimpactdetectron/wheelz/wheelz
Processing /kaggle/input/nflimpactdetectron/detectron2/detectron2
Processing /kaggle/input/nflimpactdetectron/wheelz/wheelz/fvcore-0.1.2-py3-none-any.whl
Processing /kaggle/input/nflimpactdetectron/wheelz/wheelz/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
  Created wheel for detectron2: filename=detectron2-0.3-cp37-cp37m-linux_x86_64.whl size=5563546 sha256=5dccf6cadb03101ffdb8a78bc4d17ad2a2e0e56dbb412f12f2a4b0f5217d59f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rznuls29/wheels/a8/a2/bd/c673fab154eab56fe2c82c66f310d2a33bc4a1aaefaa8182e2
Successfully built detectron2


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import cv2
import pickle

In [3]:
from torchvision.ops import box_area, box_iou

In [4]:
import sys
sys.path.append('../input/footballhelmetdetector')
from model import get_predictor
# \, new_model_cfg
from train_helmet_detector import NflImpactTrainer

1.7.0 True


In [5]:
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.model_zoo import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader, build_detection_test_loader, \
    DatasetMapper
from detectron2.structures import BoxMode, Instances, Boxes
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.structures import Instances

from detectron2.modeling.roi_heads.roi_heads import StandardROIHeads
from detectron2.modeling import ROI_HEADS_REGISTRY

from typing import Dict, List, Optional, Tuple


In [6]:
from detectron2.utils.visualizer import Visualizer

In [7]:
from copy import deepcopy 
from tqdm import tqdm

In [8]:
def convert_video_labels(video_labels: pd.DataFrame):
    dataset_dicts = []
    all_videos = video_labels['video'].unique()
    videos_processed = 0
    for video in all_videos:
        frames = video_labels.query("video == @video")
        min_frame = frames['frame'].min()
        max_frame = frames['frame'].max()

        for i in range(min_frame, max_frame + 1):
            player_frames = frames.query("frame == @i")
            vid = video.split('.')[:-1][0]
            img_name = video.split('.')[:-1][0] + '_' + str.format("{}", i).zfill(3) + '.png'
            img_path = os.path.join("../input/nfl-impact-detection-train-frames", vid, img_name)
            if not os.path.exists(img_path):
                continue
            record = {"image_id": img_path, "file_name": img_path, "height": 720, "width": 1280}
            objects = []
            if not player_frames.empty:
                for _, row in player_frames.iterrows():
                    category_id = row['impact']
#                     print(f"_ = {_}")
#                     print(f"row = {row}")
                    obj = {
                        "bbox": [row['left'],
                                 row['top'],
                                 row['left'] + row['width'],
                                 row['top'] + row['height']],
                        "bbox_mode": BoxMode.XYXY_ABS,                        
                        "category_id": 0,
                    }
                    objects.append(obj)

            record["annotations"] = objects
            if objects:
                dataset_dicts.append(record)

        print("conversion progress %d/%d" % (videos_processed, len(all_videos)))
        videos_processed += 1
    return dataset_dicts

In [9]:
# @ROI_HEADS_REGISTRY.register()
# class Nfl_crop_ROIHeads(StandardROIHeads):
#     def _convert_tl_br(self, w,h,magnitude, x1,y1,x2,y2,):
#         assert magnitude >0, 'increase size <0'

#         w_box = abs(x2 - x1)*magnitude
#         assert w_box >0, 'w_box <0'
#         h_box = abs(y2 - y1)*magnitude
#         assert h_box >0, 'h_box <0'

#         x1 = x1 - w_box if (x1 - w_box) >0 else 0
#         y1 = y1 - h_box if (y1 - h_box) >0 else 0

#         x2 = x2 + w_box if (x2 + w_box) <w else w
#         y2 = y2 + h_box if (y2 + h_box) <h else h

# #         return torch.tensor([x1,y1,x2,y2]).to('cuda')
#         return torch.tensor([x1,y1,x2,y2])

    
    
#     def transform_proposals(self, proposals: List[Instances], magnitude: int):
# #         proposals = deepcopy(input_proposals)
# #         assert len(proposals) == 1, 'len of proposal != 1'
#         for instance in proposals:
#             w, h = instance.image_size
#             tensor = instance.get('proposal_boxes')
#     #         temp = tensor.apply_(lambda x: convert_tl_br(720,1280,magnitude,*x))
#             transform_boxes = []
#             for i in tensor:    
#                 transform_boxes.append(self._convert_tl_br(w,h,magnitude,*i))
#             transform_boxes = torch.stack(transform_boxes)
#             instance.set('proposal_boxes', Boxes(transform_boxes))
#         return proposals

#     def _forward_box(self, features: Dict[str, torch.Tensor], proposals: List[Instances]):
#         """
#         Forward logic of the box prediction branch. If `self.train_on_pred_boxes is True`,
#             the function puts predicted boxes in the `proposal_boxes` field of `proposals` argument.

#         Args:
#             features (dict[str, Tensor]): mapping from feature map names to tensor.
#                 Same as in :meth:`ROIHeads.forward`.
#             proposals (list[Instances]): the per-image object proposals with
#                 their matching ground truth.
#                 Each has fields "proposal_boxes", and "objectness_logits",
#                 "gt_classes", "gt_boxes".

#         Returns:
#             In training, a dict of losses.
#             In inference, a list of `Instances`, the predicted instances.
#         """
#         features = [features[f] for f in self.box_in_features]
#         #          
#         proposals = self.transform_proposals(proposals, 1)
#         box_features = self.box_pooler(features, [x.proposal_boxes for x in proposals])
#         box_features = self.box_head(box_features)
#         predictions = self.box_predictor(box_features)
#         del box_features

#         if self.training:
#             losses = self.box_predictor.losses(predictions, proposals)
#             # proposals is modified in-place below, so losses must be computed first.
#             if self.train_on_pred_boxes:
#                 with torch.no_grad():
#                     pred_boxes = self.box_predictor.predict_boxes_for_gt_classes(
#                         predictions, proposals
#                     )
#                     for proposals_per_image, pred_boxes_per_image in zip(proposals, pred_boxes):
#                         proposals_per_image.proposal_boxes = Boxes(pred_boxes_per_image)
#             return losses
#         else:
#             pred_instances, _ = self.box_predictor.inference(predictions, proposals)
#             return pred_instances

In [10]:
!mkdir output

In [11]:
# def new_model_cfg(custom_ROIHeads=None):
#     cfg = get_cfg()
#     model = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
#     cfg.merge_from_file(model_zoo.get_config_file(model))
# #     cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    
#     cfg.MODEL.MASK_ON = False
#     cfg.INPUT.RANDOM_FLIP = "none"
#     cfg.OUTPUT_DIR = "output"

#     cfg.DATALOADER.NUM_WORKERS = 2
#     cfg.SOLVER.IMS_PER_BATCH = 4
#     cfg.SOLVER.CHECKPOINT_PERIOD = 2000
#     cfg.SOLVER.STEPS = (21000, 50000)
#     cfg.SOLVER.MAX_ITER = 10000
#     cfg.SOLVER.BASE_LR = 0.001
#     cfg.TEST.EVAL_PERIOD = 2000
#     cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
#     cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
#     if(custom_ROIHeads):
# #         cfg.MODEL.ROI_HEADS.NAME = 'Nfl_crop_ROIHeads'
#         cfg.MODEL.ROI_HEADS.NAME = custom_ROIHeads
    
#     cfg.MODEL.WEIGHTS = "../input/nflimpactdetectron/helmet_detector_model_0093999.pth"
#     return cfg

def new_model_cfg():
    cfg = get_cfg()
    model = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
    cfg.merge_from_file(model_zoo.get_config_file(model))
#     cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    cfg.MODEL.WEIGHTS = "../input/nflimpactdetectron/helmet_detector_model_0093999.pth"
    cfg.MODEL.MASK_ON = False
    cfg.INPUT.RANDOM_FLIP = "none"
    cfg.OUTPUT_DIR = "output"

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.CHECKPOINT_PERIOD = 2000
    cfg.SOLVER.STEPS = (21000, 50000)
    cfg.SOLVER.MAX_ITER = 1000
    cfg.SOLVER.BASE_LR = 0.001
    cfg.TEST.EVAL_PERIOD = 2000
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
        
    return cfg

# Prepare train and test

In [12]:
# path = '../input/nfl-csv/Full_4frame_impact.csv'

# video_labels = pd.read_csv(path, index_col=0)
# video_labels = video_labels[video_labels['impact']>0]

# train_gameKey, test_gameKey = train_test_split(video_labels['gameKey'].unique(), test_size= 0.2)

# def check_split(x):
#     if x['gameKey'] in test_gameKey:
#         return 'Test' 
#     else: return 'Train'

# video_labels['split'] = video_labels.apply(lambda x: check_sp lit(x), axis=1)

# train_df = video_labels[(video_labels['split'] == 'Train') & (video_labels['impact'] == 2)]
# test_df = video_labels[(video_labels['split'] == 'Test') & (video_labels['impact'] == 2)]
# len(train_df.groupby('image_name'))
# len(test_df.groupby('image_name'))
# train = convert_video_labels(train_df)
# test = convert_video_labels(test_df)
# with open('e2e_impact2.pickle', 'wb') as f:
#     pickle.dump((train, test), f, pickle.HIGHEST_PROTOCOL)

In [13]:
# im = cv2.imread('../input/nfl-impact-detection-train-frames/57583_000082_Endzone/57583_000082_Endzone_001.png')
# dataset_dicts = convert_video_labels(video_labels)
# dataset_dicts = pd.read_pickle('../input/nfl-csv/dataset_dicts.pickle')
# dataset_dicts = pd.read_pickle('../input/nfl-csv/detectron2_4frame_dataset.pickle')

# train, test = train_test_split(dataset_dicts, test_size=0.05, shuffle=False)
# train = convert_video_labels(df[df['split']=='Train'])
# test = convert_video_labels(df[df['split']=='Test'])
train, test = pd.read_pickle('../input/nfl-csv/e2e_impact2.pickle')

DatasetCatalog.register("nflimpact_train", lambda: train)
DatasetCatalog.register("nflimpact_test", lambda: test)

classes = ["impact"]
# classes = ["helmet"]
# classes = ["helmet", "semi_impact", "impact"]
MetadataCatalog.get("nflimpact_train").set(thing_classes=classes)
MetadataCatalog.get("nflimpact_test").set(thing_classes=classes)

cfg = new_model_cfg()

# assert cfg.MODEL.ROI_HEADS.NAME == 'NflROIHeads', 'incorrect ROI_HEADS.NAME'

cfg.DATASETS.TRAIN = ("nflimpact_train",)
cfg.DATASETS.TEST = ("nflimpact_test",)


In [14]:
cfg.SOLVER.MAX_ITER = 10000
print(cfg.MODEL.WEIGHTS)
trainer = NflImpactTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

../input/nflimpactdetectron/helmet_detector_model_0093999.pth


/opt/conda/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:217: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


Loading and preparing results...
DONE (t=0.06s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.65s).
Accumulating evaluation results...
DONE (t=0.53s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.357
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.142
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.484
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.495
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [15]:
# im = cv2.imread('../input/nfl-impact-detection-train-frames/57583_000082_Endzone/57583_000082_Endzone_001.png')
# height, width = im.shape[:2]
# image = torch.as_tensor(im.astype("float32").transpose(2, 0, 1))
# inputs = [{"image": image, "height": height, "width": width }]
# # 'image_sizes':(480,640)}
# cfg.MODEL.DEVICE ='cpu'
# model = build_model(cfg)

# # trainer = NflImpactTrainer(cfg)
# DetectionCheckpointer(model).load('../input/nflimpactdetectron/helmet_detector_model_0093999.pth')


# model.eval()#make sure its in eval mode
# #run model
# with torch.no_grad():
#     images = model.preprocess_image(inputs) 
#     features = model.backbone(images.tensor)
#     proposals, _ = model.proposal_generator(images, features)
# #     outter_proposals = transform(proposals)
#     instances, _ = model.roi_heads(images, features, proposals)
